In [93]:
import advent

data = advent.get_lines(23)

# Strategy: just brute force lmao
# We need some kind of limit on 'forward moves', e.g.:
# only allow moving from hole to spot or from spot to hole
# although unfortunately this restriction is probably too heavy for part 2

weights = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}
weights_rev = {1: 'A', 10: 'B', 100: 'C', 1000: 'D'}

def get_holes(data):
    # Returns something like [['C', 'C'], 3 other holes...]
    tophole, bottomhole = data[2], data[3]
    return [(weights[tophole[i]], weights[bottomhole[i]]) for i in [3, 5, 7, 9]]

def pp(pos):
    if pos is None: return '.'
    return weights_rev[pos]

def to_pretty(f):
    f = ['#############',
         '#' + pp(f[0]) + pp(f[1]) + '.' + pp(f[3]) + '.' + pp(f[5]) + '.' + pp(f[7]) + '.' + pp(f[9]) + pp(f[10]) + '#',
         '###' + pp(f[2][0]) + '#' + pp(f[4][0]) + '#' + pp(f[6][0]) + '#' + pp(f[8][0]) + '###',
         '  #' + pp(f[2][1]) + '#' + pp(f[4][1]) + '#' + pp(f[6][1]) + '#' + pp(f[8][1]) + '#  ',
         '  #########  ']
    for l in f:
        print(l)

holes = get_holes(data)
field = [None, None, holes[0], None, holes[1], None, holes[2], None, holes[3], None, None]
to_pretty(field)
# always: first position is the top, second position is bottom

#############
#...........#
###C#A#B#D###
  #C#A#D#B#  
  #########  


In [113]:
def is_hallway(spot):
    return not isinstance(spot,  tuple)

def get_all_hallways_ix():
    return [0, 1, 3, 5, 7, 9, 10]

def get_all_holes_ix():
    return [2, 4, 6, 8]

def is_empty(spot):
    return spot is None if is_hallway(spot) else all([s is None for s in spot])

def is_available(spot):
    return spot is None if is_hallway(spot) else any([s is None for s in spot])

def free_spaces_in_hole(hole):
    return sum([s is None for s in hole])    

def is_hole_available(hole, ant):
    return is_empty(hole) or hole[1] == ant

def can_hole_be_moved_from(hole, ant):
    return (hole[0] != ant and hole[0] is not None) or (hole[1] != ant and hole[1] is not None)

def path_free(from_, to, field):
    # this only checks spots, holes are never an obstruction (this does not test if `is_hole_available`)
    if from_ > to:
        from_, to = to, from_
    return all([(field[s] is None) for s in [1, 3, 5, 7, 9] if (s > from_ and s < to)])

def get_ant(spot):
    # Get the top ant, or None if there are no ants
    return spot if is_hallway(spot) else (spot[1] if spot[0] is None else spot[0])

def get_hole_ix(ant):
    # given ant letter, return index on the field of the hole. hardcoded yeah!
    return {1: 2, 10: 4, 100: 6, 1000: 8}[ant]

def get_ant_ix(hole_ix):
    return {2: 1, 4: 10, 6:100, 8: 1000}[hole_ix]

def is_solved(field):
    return field[2] == (1, 1) and field[4] == (10, 10) and field[6] == (100, 100) and field[8] == (1000, 1000)

# We represent a move as (i, j), meaning a move from the i'th field position to the j'th
# if i is a hole, we move the topmost ant, if j is a hole, we move to the bottommost available position
def get_moves(field):
    moves = []
    # Only allow moves from hole to spot and vice-versa
    # do not allow moving from a hole if the hole only contains correct ants (to prevent infinite loops)
    for spot_ix in get_all_hallways_ix():
        spot = field[spot_ix]
        if is_empty(spot): continue # no move to be made with this origin
        ant = spot # cannot be none
        hole_ix = get_hole_ix(ant)
        hole = field[hole_ix]
        if is_hole_available(hole, ant) and path_free(spot_ix, hole_ix, field):
            # basically, moving into a hole is always optimal, so just return a single move
            return [(spot_ix, hole_ix, ant, False, True)]
    
    for hole_ix in get_all_holes_ix():
        hole = field[hole_ix]
        ant_ix = get_ant_ix(hole_ix)
        if not can_hole_be_moved_from(hole, ant_ix): continue
        # we now know the hole is not empty, so we can get the ant in there
        # first check if the correct hole is free, if so move it there immediately
        ant = get_ant(hole)
        target_hole_ix = get_hole_ix(ant)
        if is_hole_available(field[target_hole_ix], ant) and path_free(hole_ix, target_hole_ix, field):
            return [(hole_ix, target_hole_ix, ant, True, True)]

        for hallway_ix in get_all_hallways_ix():
            if is_empty(field[hallway_ix]) and path_free(hole_ix, hallway_ix, field):
                moves += [(hole_ix, hallway_ix, ant, True, False)]
        #return moves # in this case, we prefer moving from left most hole only!!!!!!! WRONG !!!!!
    
    return moves

def get_weight(move, field):
    # move is always from hole to hallway or vice versa
    from_, to, ant, from_hole, to_hole = move
    from_spaces = free_spaces_in_hole(field[from_]) + 1 if from_hole else 0
    to_spaces = free_spaces_in_hole(field[to]) if to_hole else 0
    return (abs(to - from_) + from_spaces + to_spaces) * ant

def get_total_weight(solution, field):
    res = 0
    for move in solution:
        res += get_weight(move, field)
        field = make_move(move, field)
    return res

def make_move(move, field_old):
    field = field_old.copy()
    from_, to, ant, fromhole, tohole = move
    if tohole:
        if field[to][1] is None:
            field[to] = (None, ant)
        else:
            field[to] = (ant, field[to][1])
    else:
        field[to] = ant
    
    if fromhole:
        if field[from_][0] is None:
            field[from_] = (None, None)
        else:
            field[from_] = (None, field[from_][1])
    else:
        field[from_] = None
    return field

# example: {fieldstr: (moveslist, weight)}, where moveslist/weight is the best found so far
cache = {}
BEST_GLOBAL_SOLUTION = 999_999_999_999  # impossibly high number

def find_best_solution(field, moves_list=[], weight_so_far=0):
    # Return the best solution found (lowest weight), or (None, x) if there is no (best) solution possible from this position
    #print(field)
    global BEST_GLOBAL_SOLUTION
    h = '-'.join(str(i) for i in field) # hash
    if weight_so_far >= BEST_GLOBAL_SOLUTION:
        return (None, -1)
    if h in cache:
        _, w = cache[h]
        if weight_so_far > w:
            return (None, -1) # We are not in an optimal path, just return impossible

    if is_solved(field):
        print(f"found a solution with weight {weight_so_far}")
        BEST_GLOBAL_SOLUTION = weight_so_far
        return (moves_list, weight_so_far)

    moves = get_moves(field)
    best_solution = (None, BEST_GLOBAL_SOLUTION)
    for move in moves: 
        field_new = make_move(move, field)
        solution = find_best_solution(field_new, moves_list + [move], weight_so_far + get_weight(move, field))
        if solution[0] is None: continue  # found no solutions
        if solution[1] < best_solution[1]:
            best_solution = solution
    return best_solution

In [114]:
best_solution = find_best_solution(field)
best_solution

found a solution with weight 22150
found a solution with weight 18150
found a solution with weight 18130
found a solution with weight 14150
found a solution with weight 12150
found a solution with weight 12130
found a solution with weight 12030
found a solution with weight 11964
found a solution with weight 11944
found a solution with weight 11830
found a solution with weight 11788
found a solution with weight 11772
found a solution with weight 11630
found a solution with weight 11566
found a solution with weight 11546
found a solution with weight 11536


([(4, 0, 1, True, False),
  (4, 1, 1, True, False),
  (6, 4, 10, True, True),
  (2, 3, 100, True, False),
  (8, 9, 1000, True, False),
  (8, 4, 10, True, True),
  (9, 8, 1000, False, True),
  (6, 8, 1000, True, True),
  (3, 6, 100, False, True),
  (2, 6, 100, True, True),
  (1, 2, 1, False, True),
  (0, 2, 1, False, True)],
 11536)

In [115]:
fieldx = field.copy()
for move in best_solution[0]:
    w = get_weight(move, fieldx)
    to_pretty(fieldx)
    print(fieldx, move, w)
    fieldx = make_move(move, fieldx)
print(fieldx)

#############
#...........#
###C#A#B#D###
  #C#A#D#B#  
  #########  
[None, None, (100, 100), None, (1, 1), None, (10, 1000), None, (1000, 10), None, None] (4, 0, 1, True, False) 5
#############
#A..........#
###C#.#B#D###
  #C#A#D#B#  
  #########  
[1, None, (100, 100), None, (None, 1), None, (10, 1000), None, (1000, 10), None, None] (4, 1, 1, True, False) 5
#############
#AA.........#
###C#.#B#D###
  #C#.#D#B#  
  #########  
[1, 1, (100, 100), None, (None, None), None, (10, 1000), None, (1000, 10), None, None] (6, 4, 10, True, True) 50
#############
#AA.........#
###C#.#.#D###
  #C#B#D#B#  
  #########  
[1, 1, (100, 100), None, (None, 10), None, (None, 1000), None, (1000, 10), None, None] (2, 3, 100, True, False) 200
#############
#AA.C.......#
###.#.#.#D###
  #C#B#D#B#  
  #########  
[1, 1, (None, 100), 100, (None, 10), None, (None, 1000), None, (1000, 10), None, None] (8, 9, 1000, True, False) 2000
#############
#AA.C.....D.#
###.#.#.#.###
  #C#B#D#B#  
  #########  
[1, 1, (N